# Derive Snow Depth From ICESat-2 ATL08 and 3DEP at Grand Mesa, CO  
Notebook developed by Hannah Besso, adapted from code by Ben Smith and containing code from David Shean for the UW Hackweek 2021

### This notebook does the following:
* Read in ATL08 data for from openAltimetry using IcyPyx, select a track to analyze, and convert to a geodataframe  
* Import 3DEP file, as downloaded from the download_3DEP.ipynb notebook  
* Extract 3DEP elevations at ATL08 locations
* Account for datum differences between ATL08 and 3DEP elevation products
* Calculate and plot snow depth by differencing the snow-on ATL08 elevations from the snow-off 3DEP elevations
* Determine mean snow depth along the chosen track

In [ ]:
# Imports necessary packages:
import numpy as np
import matplotlib.pyplot as plt
import re
import geopandas as gpd
import os
import pandas as pd
import rioxarray as rxr
from rasterio.enums import Resampling
import s3fs
import requests
import icepyx as ipx
from shapely.geometry import Point

import rasterio as rio
import rasterio.plot
from rasterio.plot import plotting_extent

from scipy.interpolate import RectBivariateSpline

In [ ]:
#%matplotlib widget
%matplotlib inline

## Read ATL08 from Open Altimetry

This section contains code written by Ben Smith for reading ATL08 data for Grand Mesa from openAltimetry.

### Import the Sentinel basemap
The sentinel basemap is a useful way to visualize map locations on the fly. This notebook does not use the Sentinel basemap very much, but this code is useful to have for future projects, and the lonlims and latlims variables created in this cell are necessary for the CMR query in the next step.

In [ ]:
# GDAL environment variables to efficiently read remote data
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' 
os.environ['AWS_NO_SIGN_REQUEST']='YES' 

# SAR Data are stored in a public S3 Bucket
url = 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/12/S/YJ/2016/S1B_20161121_12SYJ_ASC/Gamma0_VV.tif'

# These Cloud-Optimized-Geotiff (COG) files have 'overviews', low-resolution copies for quick visualization
XR=[725000.0, 767000.0]
YR=[4.30e6, 4.34e6]
# open the dataset
da = rxr.open_rasterio(url, overview_level=1).squeeze('band')#.clip_box([712410.0, 4295090.0, 797010.0, 4344370.0])
da=da.where((da.x>XR[0]) & (da.x < XR[1]), drop=True)
da=da.where((da.y>YR[0]) & (da.y < YR[1]), drop=True)
dx=da.x[1]-da.x[0]
SAR_extent=[da.x[0]-dx/2, da.x[-1]+dx/2, np.min(da.y)-dx/2, np.max(da.y)+dx/2]

# Prepare coordinate transformations into the basemap coordinate system
from pyproj import Transformer, CRS
crs=CRS.from_wkt(da['spatial_ref'].spatial_ref.crs_wkt)
to_image_crs=Transformer.from_crs(crs.geodetic_crs, crs)
to_geo_crs=Transformer.from_crs(crs, crs.geodetic_crs)

corners_lon, corners_lat=to_geo_crs.transform(np.array(XR)[[0, 1, 1, 0, 0]], np.array(YR)[[0, 0, 1, 1, 0]])
lonlims=[np.min(corners_lat), np.max(corners_lat)]
latlims=[np.min(corners_lon), np.max(corners_lon)]

### Use IcePyx to query CMR (NASA EOSDIS Common Metadata Repository) for available granules

(Can probably do this with openAltimetry too)

In [ ]:
region_a = ipx.Query('ATL08', [lonlims[0], latlims[0], lonlims[1], latlims[1]], ['2020-01-01','2020-6-30'], \
                          start_time='00:00:00', end_time='23:59:59')

In [ ]:
region_a.avail_granules()

In [ ]:
ATLAS_re=re.compile('ATL.._(?P<year>\d\d\d\d)(?P<month>\d\d)(?P<day>\d\d)\d+_(?P<track>\d\d\d\d)')

date_track=[]
for count, item in enumerate(region_a.granules.avail):
    granule_info=ATLAS_re.search(item['producer_granule_id']).groupdict()
    date_track += [ ('-'.join([granule_info[key] for key in ['year', 'month', 'day']]), granule_info['track'])]

### Define a function to read ATL08 from OpenAltimetry

Uses the latlim and lonlim coordinates, along with the date and track information from the above cell to request data from OpenAltimetry. The output (as specified in the function notes) is a dictionary of the data, with individual tracks, dates, and variables as keys.

In [ ]:
def get_OA_ATL08(date_track, lonlims, latlims, beamnames=["gt1l","gt1r","gt2l","gt2r","gt3l","gt3r"]):
    '''
    retrieve ICESat2 ATL08 data from OpenAltimetry
    
    Inputs:
        date_track: a list of tuples.  Each contains a date string "YYYY-MM-DD" and track number (4-character string)
        lonlims: longitude limits for the search
        latlims: latitude limits for the search
        beamnames: list of strings for the beams
    outputs:
        a dict containing ATL08 data by beam name
    
    Due credit:
        Much of this code was borrowed Philipp Arndt's Pond Picker repo: https://github.com/fliphilipp/pondpicking
    '''
      
    IS2_data={}
    for this_dt in date_track:
        this_IS2_data={}
        for beamname in beamnames:
            oa_url = 'https://openaltimetry.org/data/api/icesat2/atl08?minx={minx}&miny={miny}&maxx={maxx}&maxy={maxy}&trackId={trackid}&beamName={beamname}&outputFormat=json&date={date}&client=jupyter'
            oa_url = oa_url.format(minx=lonlims[0],miny=latlims[0],maxx=lonlims[1], maxy=latlims[1], 
                                   trackid=this_dt[1], beamname=beamname, date=this_dt[0], sampling='true')
            #.conf_ph = ['Noise','Buffer', 'Low', 'Medium', 'High']
            if True:
                r = requests.get(oa_url)
                data = r.json()
                D={}
                D['lat_seg'] = []
                D['lon_seg'] = []
                D['h_te_best_fit'] = []
                D['h_canopy']=[]
                for series in data['series']:
                    for p in series['lat_lon_elev_canopy']:
                        D['lat_seg'].append(p[0])
                        D['lon_seg'].append(p[1])
                        if p[2] is not None:
                            D['h_te_best_fit'].append(p[2])
                        else:
                            D['h_te_best_fit'].append(np.NaN)
                        if p[3] is not None:
                            D['h_canopy'].append(p[3])
                        else:
                            D['h_canopy'].append(np.NaN)
                D['x_seg'], D['y_seg']=to_image_crs.transform(D['lat_seg'], D['lon_seg'])
                for key in D:
                    D[key]=np.array(D[key])
                if len(D['lat_seg']) > 0:
                    this_IS2_data[beamname]=D
            #except Exception as e:
            #    print(e)
            #    pass
        if len(this_IS2_data.keys()) > 0:
            IS2_data[this_dt] = this_IS2_data
    return IS2_data

### Read the ATL08 Data from Open Altimetery
And choose a single track to analyze, converting into a geodataframe for easier analysis

In [ ]:
# Call the function to read the data
D08 = get_OA_ATL08(date_track, lonlims, latlims)

In [ ]:
# March 11 is the date closest to the SnowEx field campaign (late January through Feb 14) with ICESat-2 data intersecting Grand Mesa.
# Use the date and track keys to pull out the data for March 11:

march_11 = D08['2020-03-11','1156']

In [ ]:
# Check that the ICESat-2 data we chose does in fact intersect Grand Mesa by plotting it with the Sentinel basemap

plt.figure()
plt.imshow(np.array(da)[::-1,:], origin='lower', extent=SAR_extent, cmap='gray', clim=[0, 0.5])#plt.figure();


for beam, D in march_11.items():
    plt.plot(D['x_seg'], D['y_seg'], '.', markersize=3)

In [ ]:
# Convert to a geodataframe for easier analysis, using the UTM Zone 12N projection for Grand Mesa

gdf = gpd.GeoDataFrame(march_11['gt1l'], crs='EPSG:32612', geometry=gpd.points_from_xy(march_11['gt1l']['x_seg'], march_11['gt1l']['y_seg'],))

In [ ]:
# Get a first look at the data
gdf.head()

### Clip the ATL08 data to only the flat top of the Mesa

We don't want to calculate snow depth for the steeply sloping areas surrounding the Mesa.

In [ ]:
# Read in a polygon of only the flat top of the Mesa

poly_fn = "/home/jovyan/space_lasers/notebooks/GM_tight_poly_32612.geojson"
poly_gm = gpd.read_file(poly_fn)

In [ ]:
# Clip the ATL08 geodataframe to the flat top of the Mesa:

march_11_clip = gpd.clip(gdf, poly_gm)

## Import 3DEP data
Use the tiled 3DEP tif created in the download_3DEP.ipynb notebook here.

In [ ]:
# Use the file path for the tiff created in the download_3DEP.ipynb notebook:

tif_fn = '/home/jovyan/space_lasers/notebooks/gm_3dep_1m_lidar_tiles.tif'

In [ ]:
# Open the 3DEP tiff and rescale it from 1m resolution to 3m resolution to speed up computation time 
# (and to make it possible to work with, under the 8GB limit on the Hackweek JupyterHub!)
# Also reproject to match the ATL08 data projection of UTM Zone 12N

lidar_ds=rxr.open_rasterio(tif_fn)
#resample the DTM to ~3m:
scale_factor = 1/3
new_width = int(lidar_ds.rio.width * scale_factor)
new_height = int(lidar_ds.rio.height * scale_factor)

#reproject the horizontal CRS to match ICESat-2
UTM_wgs84_crs=CRS.from_epsg(32612)
lidar_3m = lidar_ds.rio.reproject(
    UTM_wgs84_crs,
    shape=(new_height, new_width),
    resampling=Resampling.bilinear,
)

In [ ]:
# Plot the data

plt.figure(); 
lidar_3m.sel(band=1).plot.imshow(vmin=2900, vmax=3300)

### Extract 3DEP Snow-Off Elevations at ATL08 Snow-On Locations

In [ ]:
# Create a denser grid of the 3DEP data
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.RectBivariateSpline.html

interpolator = RectBivariateSpline(np.array(lidar_3m.y)[::-1], np.array(lidar_3m.x), 
                                   np.array(lidar_3m.sel(band=1))[::-1,:], kx=1, ky=1)

In [ ]:
x0=np.array(lidar_3m.x)
y0=np.array(lidar_3m.y)


ii = (march_11_clip['x_seg'] > np.min(x0)) & (march_11_clip['x_seg'] < np.max(x0))

ii &= (march_11_clip['y_seg'] > np.min(y0)) & (march_11_clip['y_seg'] < np.max(y0))

# Evaluate the spline at the ATL08 point locations, making an ndarray of the 3DEP elevations at the ATL08 point locations
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.RectBivariateSpline.ev.html
zi_1l = interpolator.ev(march_11_clip['y_seg'][ii], march_11_clip['x_seg'][ii])

In [ ]:
# Get rid of invalid values (in this case they are -9999)

zi_1l[zi_1l <= 0] = np.nan

In [ ]:
# Account for the datum offset between the datasets by subtracing 15.6 m from the 3DEP elevations

zi_1l = zi_1l - 15.6

### Plot the location of the ICESat-2 track and the elevation of the snow-off and snow-on datasets

In [ ]:
fig=plt.figure(figsize=[8, 5]); 
hax=fig.subplots(1,2)
plt.sca(hax[0])
lidar_3m.sel(band=1).plot.imshow(vmin=2900, vmax=3300)
plt.plot(march_11_clip['x_seg'][ii], march_11_clip['y_seg'][ii],'.', c='orange')
plt.axis('equal')
plt.title('Grand Mesa ICESat-2 Track')

plt.sca(hax[1])
plt.plot(march_11_clip['y_seg'][ii], march_11_clip['h_te_best_fit'][ii],'.', label='March 11')
plt.plot(march_11_clip['y_seg'][ii], zi_1l,'.', label='DTM')
plt.legend()
plt.title('ATL08 GT1L')
plt.xlabel('x coordinate of projection [meter]')
plt.ylabel('Elevation (m)')
plt.tight_layout()
#plt.savefig('ATL08_gt1l.jpeg', dpi=300)

### Calculate Snow Depth by subtracting the snow off from the snow on data

In [ ]:
dif_1l = march_11_clip['h_te_best_fit'] - zi_1l

In [ ]:
dif1l = dif_1l.to_frame()
dif1l.head(5)

In [ ]:
# Remove the first data point, which appears to be an outlier (perhaps too close to the edge of the mesa)

dif_1l = dif_1l[1:]

In [ ]:
# Investigate the basic statistics of the snow depth dataset

dif_1l.describe()

### Plot the Snow Depth at each point, and the mean snow depth along the ICESat-2 track

In [ ]:
fig, ax = plt.subplots()
ax.scatter(dif_1l.index, dif_1l)
ax.axhline(y=0, c = 'black', linestyle='-')
ax.axhline(y=0.352486, c = 'grey', linestyle='--')
ax.set_xlabel('Along Track Distance')
ax.set_ylabel('Snow Depth (m)')
ax.set_title('Snow Depth (m) Derived From ICESat-2 and 3DEP')
#plt.savefig('SnowDepthScatter.jpeg', dpi=300)